In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  3 10:25:44 2020
Updated on Thur Jun 24 14:39:00 2021

@author: KStanely



Code created to scrape, download and process UNFCCC CRF data.

Functions included in package: 
    unfccc:
        - Overarching script that scrapes UNFCCC data, unzips the data, processes
          data into a common format by gas and by country and plots data by region
          or country. Function calls:
            - unfccc_scrape
            - unfccc_unzip
            - crf_import
            - crf_plot
        - Requires:
            - year = the year (as a string) of the data to scrape.
            - data_path = a base directory of where the data will be stored.
            - scrape = True/False depending if you want to (re)scrape data from
                       UNFCCC website and store zip files in
                       data_path/year/data/zip/.
            - unzip = True/False depending if you want to (re-)unzip downloaded
                      zip files in data_path/year/data/zip/.
            - process = True/False depending if you want to (re)process unzipped
                        files in data_path/year/data/zip/ and store in
                        data_path/year/data/.
            - plot = True/False depending if you want to (re)plot data.
            - plot_country = see crf_plot explanation for information.
            - plot_region = see crf_plot explanation for information.
        - Output is stored in data_path:
            - raw data: in data_path/year/data/zip/ by country.
            - scraped file: csv file containing all countries and urls scraped
                            from UNFCCC website in data_path/year/scraped_urls.csv.
            - unzipped data: in data_path/year/data/ by country.
            - processed data: csv files in data_path/processed_data in country files
                              for all gases (Country_NIR_YYYY_all_gases_yearly.csv)
                              and gas specific (i.e. Australia_NIR_2019_SF6.csv).
            - plots: png files in data_path/year/plots/ by country/region plotted.
    unfccc_scrape:
        - Used to scrape URLs from UNFCCC website.
        - Requires the specific version of chromdriver (line 152) that matches
          your version of Chrome. Chromedriver can be downloaded from 
          https://chromedriver.chromium.org/downloads
        - Requires:
            - year = the year (as a string) of the data to scrape.
            - data_path = a base directory of where the data will be stored
        - Output is a list of lists called: unfccc_urls
    unfccc_unzip:
        - used to unzip data folders scraped from UNFCCC website by unfccc_scrape()
        - Requires:
            - year = the year (as a string) of the data.
            - data_path = a base directory of where the data will be stored
        - Output is country specific folders with excel files in
    crf_import:
        - Used to import excel datafiles unzipped by unfccc_unzip
        - Requires:
            - year = the year (as a string) of the data.
            - data_path = a base directory of where the data will be stored
        - Output is country specific and gas specific data in excel files, stored
          in the data_path/year/processed_data/
    crf_plot:
        - Used to plot up CRF data
        - Requires:
            - year = the year (as a string) of the data to scrape.
            - data_path = a base directory of where the plots will be stored
            - plot_country = string indicating data to plot. Requires capital leter
                        at start of name and no spaces, e.g. NewZealand. For The
                        Netherlands
            - plot_regions = string indicating Annex I regions to plot data, from
                             the following:
                - all = all Annex I countries, EU27 as a total (not by individual states)
                - EU = EU27 states, excludes UK
                - EU28 = EU27 states, plus UK
                - EU+ = EU27 states, plus UK, Liechtenstein, Iceland, Monaco,
                        Norway and Switzerland
                - NAFTA = USA & Canada
                - Australasia = Australia, New Zealand & Japan
                - EITs = Economies in transition: Belarus, Croatia, Czech Republic,
                         Estonia, Hungary, Kazakhstan, Latvia, Lithuania, Poland,
                         Romania, Russia, Slovakia, Slovenia & Ukraine
                - Soviet = Former soviet states: Belarus, Estonia, Kazakhstan,
                         Latvia, Lithuania, Russia & Ukraine
        - Output is in png format and stored in the data_path/year/plots/ by
          country/region.
"""


###############################################################################
### unfccc_scrape function
###############################################################################

def unfccc(process_year = '2022', data_path = '/Users/ks15697/Documents/Python/UNFCCC/',
           scrape = True, unzip = True, process = True, plot = True,
           plot_country = "Germany", plot_region = None):

    if scrape == True:
        unfccc_scrape(year = process_year, data_path = data_path)
        input("Press Enter to continue...")

    if unzip == True:
        unfccc_unzip(year = process_year, data_path = data_path)

    if process == True:
        crf_import(year = process_year, data_path = data_path)

    if plot == True:
        crf_plot(year = process_year, data_path = data_path, plot_country = plot_country,
                 plot_region = plot_region)

###############################################################################
### unfccc_scrape function
###############################################################################

def unfccc_scrape(year = '2022', data_path = '/Users/ks15697/Documents/Python/UNFCCC/'):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    #from webdriver_manager.chrome import ChromeDriverManager
    import os
    import pandas as pd
    import glob
    import time
    
    ### country naming dictionary
    country_dict = {'ausl':'Australia', 'aus':'Australia',
                  'aut':'Austria',
                  'bel':'Belgium',
                  'bgr':'Bulgaria',
                  'blr':'Belarus',
                  'can':'Canada',
                  'che':'Switzerland',
                  'cyp':'Cyprus',
                  'cze':'CzechRepublic',
                  'deu':'Germany',
                  'dnk':'Denmark',
                  'esp':'Spain',
                  'est':'Estonia',
                  'eua':'European Union',
                  'fin':'Finland',
                  'fra':'France',
                  'frk':'France',
                  'gbk':'United Kingdom', 'gbr':'United Kingdom',
                  'grc':'Greece',
                  'hrv':'Croatia',
                  'hun':'Hungary',
                  'irl':'Ireland',
                  'isl':'Iceland',
                  'ita':'Italy',
                  'jpn':'Japan',
                  'kaz':'Kazakhstan', 'Kaz':'Kazakhstan',
                  'lie':'Liechtenstein',
                  'ltu':'Lithuania',
                  'lux':'Luxembourg',
                  'lva':'Latvia',
                  'mco':'Monaco',
                  'mlt':'Malta',
                  'nld':'Netherlands',
                  'nor':'Norway',
                  'nzl':'New Zealand',
                  'pol':'Poland',
                  'prt':'Portugal',
                  'rou':'Romania',
                  'rus':'Russian Federation',
                  'svk':'Slovakia',
                  'svn':'Slovenia',
                  'swe':'Sweden',
                  'tur':'Turkey',
                  'ukr':'Ukraine',
                  'usa':'United States'}

    ### Check if correct file structure for year exists:
    print("Checking folder structure...")
    if os.path.isdir(data_path+year) == False:
        os.mkdir(data_path+year)
        os.mkdir(data_path+year+"\\data")
        os.mkdir(data_path+year+"\\data\\zip")
    elif os.path.isdir(data_path+year+"\\data") == False:
        os.mkdir(data_path+year+"\\data")
        os.mkdir(data_path+year+"\\data\\zip")
    else:
        print("...Folder structure correct")
    zip_dir = data_path+year+r'/data/zip/'

    
    print("\nScaping ",year," UNFCCC country CRF URLs...")

    ### set website to look at:
    #website = "https://unfccc.int/process-and-meetings/transparency-and-reporting/reporting-and-review-under-the-convention/greenhouse-gas-inventories-annex-i-parties/national-inventory-submissions-"+year
    website = "https://unfccc.int/ghg-inventories-annex-i-parties/"+year

    ### Setup system to run the web scraping
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    prefs = {}
    prefs["profile.default_content_settings.popups"]=0
    prefs["download.default_directory"]='C:\\Users\\amar1\\UNFCCC_data_local\\2022\\data\\zip\\'#zip_dir
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
    driver = webdriver.Chrome(executable_path= r'C:/Users/amar1/Downloads/chromedriver_win32/chromedriver.exe',
                              chrome_options=chrome_options)

    ### Scrape website
    driver.get(website)


    results = driver.find_elements("css selector","tbody")

    # create empty array to store data
    unfccc_urls = []
    # loop over results
    for result in results:
        lines = result.find_elements("tag name" ,"tr")
        for line in lines:
            temp = []
            country = line.find_elements("tag name" ,"th")[0].text
            temp.append(country)
            sdate = line.find_elements("tag name" ,"td")[0].text
            if any(map(str.isdigit, sdate)) == False:
                blank = ["NaN", "NaN", "NaN"]
                temp.extend(blank)
            else:
                temp.append(sdate)
                crf_check = line.find_elements("tag name" ,"a")[0].text
                if len(crf_check) > 3:
                    crf_check_temp = crf_check.split(" ")
                    crf_check = crf_check_temp[0]
                if crf_check == 'CRF':
                    link = line.find_elements("tag name" ,"a")[0]
                else:
                    for ii in range(0,10):
                        crf_check = line.find_elements("tag name" ,"a")[ii].text
                        if len(crf_check) > 3:
                            crf_check_temp = crf_check.split(" ")
                            crf_check = crf_check_temp[0]
                        link = line.find_elements("tag name" ,"a")[ii]
                        if crf_check == 'CRF(Convention)': break
                        elif crf_check == 'CRF': break
                temp.append(crf_check)
                base_link = link.get_attribute("href")
                temp.append(base_link)
                # append dict to array
            unfccc_urls.append(temp)

    # close driver
    driver.quit()

    print(year," UNFCCC country CRF URLs scraped!!!")


    ### get url of zip files for UNFCCC CRF data
    print("\nScaping individual countries ",year," CRF zip file URLs...")

    for i in range(len(unfccc_urls)):
        if unfccc_urls[i][1] != "NaN":
            print("\nScraping", unfccc_urls[i][0], "UNFCCC CRF URL...")
            driver = webdriver.Chrome(executable_path= r'C:\Users\amar1\Downloads\chromedriver_win32\chromedriver.exe',   chrome_options=chrome_options)
            driver.get(unfccc_urls[i][3])
            link = driver.find_element("xpath", "/html/body/div[1]/div[3]/div/section/div/article/div/div/div[2]/div[1]/div/section[1]/div/div/a")
            zip_link = link.get_attribute("href")
            # add to unfccc_urls
            unfccc_urls[i].append(zip_link)
            print(unfccc_urls[i][0], "UNFCCC CRF URL scraped!!!")
            driver.find_element("xpath", "/html/body/div[1]/div[3]/div/section/div/article/div/div/div[2]/div[1]/div/section[1]/div/div/a").click()
            print(unfccc_urls[i][0], "UNFCCC CRF zip data downloaded!!!")
        else:
            print("\nNo", unfccc_urls[i][0], "UNFCCC CRF data to scrape!")
            unfccc_urls[i].append("NaN")
        time.sleep(15)
        driver.quit()

    ### Write csv with all scraped URLs
    df = pd.DataFrame(unfccc_urls)
    df.columns = ["Country", "Submission_date", "Data_type", "Country_URL", "Data_URL"]
    df.to_csv(data_path+year+"/scraped_urls.csv")
    
    ### Check all files downloaded correctly
    print("\n...Checking",year,"scraped files...")
    all_zip_files = [f for f in glob.glob(data_path+year+"/data/zip/*[!.zip]", 
                                 recursive=False)]
    for j in range(len(all_zip_files)):
        import_country = country_dict[all_zip_files[j].split('/')[-1].split('-')[0]]
        
        for i in unfccc_urls:
            if i[0] == import_country:
                import_country_loc = i
                break
        
        print("\nRescraping", import_country, "UNFCCC CRF URL...")
        driver = webdriver.Chrome(executable_path= r'C:\Users\amar1\Downloads\chromedriver_win32\chromedriver.exe',   chrome_options=chrome_options)
        driver.get(import_country_loc[3])

        driver.find_element("xpath", "/html/body/div[1]/div[3]/div/section/div/article/div/div/div[2]/div[1]/div/section[1]/div/div/a").click()
        print(import_country, "UNFCCC CRF zip data downloaded!!!")
        
        os.remove(all_zip_files[j])

    print("\n",year,"UNFCCC CRF zip file URLs scraped!!!")




###############################################################################
### unfccc_download
###############################################################################

def unfccc_unzip(year = '2021', data_path = '/Users/ks15697/Documents/Python/UNFCCC/'):
    import os
    import glob
    from zipfile import ZipFile
    from distutils.dir_util import copy_tree

    # set the base directory
    base_dir = data_path+year+"/data/"

    ### country naming dictionary
    names_dict = {'ausl':'Australia_NIR_'+year,
                  'aus':'Australia_NIR_'+year,
                  'aut':'Austria_NIR_'+year,
                  'bel':'Belgium_NIR_'+year,
                  'bgr':'Bulgaria_NIR_'+year,
                  'blr':'Belarus_NIR_'+year,
                  'can':'Canada_NIR_'+year,
                  'che':'Switzerland_NIR_'+year,
                  'cyp':'Cyprus_NIR_'+year,
                  'cze':'CzechRepublic_NIR_'+year,
                  'deu':'Germany_NIR_'+year,
                  'dnk':'Denmark_NIR_'+year,
                  'esp':'Spain_NIR_'+year,
                  'est':'Estonia_NIR_'+year,
                  'eua':'Europe_NIR_'+year,
                  'fin':'Finland_NIR_'+year,
                  'fra':'France_NIR_'+year,
                  'frk':'France_NIR_'+year,
                  'gbk':'UK_NIR_'+year,
                  'gbr':'UK_NIR_'+year,
                  'grc':'Greece_NIR_'+year,
                  'hrv':'Croatia_NIR_'+year,
                  'hun':'Hungary_NIR_'+year,
                  'irl':'Ireland_NIR_'+year,
                  'isl':'Iceland_NIR_'+year,
                  'ita':'Italy_NIR_'+year,
                  'jpn':'Japan_NIR_'+year,
                  'kaz':'Kazakhstan_NIR_'+year,
                  'Kaz':'Kazakhstan_NIR_'+year,
                  'lie':'Liechtenstein_NIR_'+year,
                  'ltu':'Lithuania_NIR_'+year,
                  'lux':'Luxembourg_NIR_'+year,
                  'lva':'Latvia_NIR_'+year,
                  'mco':'Monaco_NIR_'+year,
                  'mlt':'Malta_NIR_'+year,
                  'nld':'Netherlands_NIR_'+year,
                  'nor':'Norway_NIR_'+year,
                  'nzl':'NewZealand_NIR_'+year,
                  'pol':'Poland_NIR_'+year,
                  'prt':'Portugal_NIR_'+year,
                  'rou':'Romania_NIR_'+year,
                  'rus':'Russia_NIR_'+year,
                  'svk':'Slovakia_NIR_'+year,
                  'svn':'Slovenia_NIR_'+year,
                  'swe':'Sweden_NIR_'+year,
                  'tur':'Turkey_NIR_'+year,
                  'ukr':'Ukraine_NIR_'+year,
                  'usa':'USA_NIR_'+year}


    ### Unzip downloaded data
    filelis = sorted([f for f in os.listdir(base_dir+"zip/") if f.endswith('.zip')])

    print("Unzipping UNFCCC CRF data...")

    for j in range(len(filelis)):
        acronym = filelis[j].split("-")
        print("... ", acronym[0])
        # Make a folder for the files
        country_path = base_dir+names_dict[acronym[0]]
        os.mkdir(country_path)
        # Create a ZipFile Object and load sample.zip in it
        zip_path = base_dir + "zip/" + filelis[j]
        with ZipFile(zip_path, 'r') as zipObj:
            # Extract all the contents of zip file in different directory
            zipObj.extractall(country_path)
        print(acronym[0], " done!")

    print("Checking data...")
    ignored = {"zip", ".DS_Store"}
    dirlis = [x for x in os.listdir(base_dir) if x not in ignored]
    for jj in dirlis:
        subdir = base_dir + jj + "/"
        subdir_test = set([os.path.dirname(subdir) for p in glob.glob(subdir +"*/*")])
        if len(subdir_test) > 0:
            print("Copying ",jj," subfolder data to correct directory...")
            copydir = os.listdir(subdir)
            copy_tree(subdir + copydir[0], subdir)


    print("All UNFCCC CRF files unzipped!!!")

###############################################################################
### crf_import_ch4
###############################################################################

def crf_import_ch4(year = '2022', data_path = '/Users/ks15697/Documents/Python/UNFCCC/'): 
    import os
    import pandas as pd
    import csv
    import numpy as np

    ### Add folder for processed data to be stored in
    if os.path.isdir(data_path+year+"/processed_data/") == False:
        os.mkdir(data_path+year+"/processed_data/")
        os.mkdir(data_path+year+"/processed_data/totals/")
        os.mkdir(data_path+year+"/processed_data/totals_co2/")
        os.mkdir(data_path+year+"/processed_data/species_specific/")
    if os.path.isdir(data_path+year+"/processed_data/totals/") == False:
        os.mkdir(data_path+year+"/processed_data/totals/")
    if os.path.isdir(data_path+year+"/processed_data/totals_co2/") == False:
        os.mkdir(data_path+year+"/processed_data/totals_co2/")
    if os.path.isdir(data_path+year+"/processed_data/species_specific/") == False:
        os.mkdir(data_path+year+"/processed_data/species_specific/")

    ### Create lists for each gas to go into for country totals

    ### Can this be changed to create lists for each YEAR (headers for Table10s3 are years rather than gases)???

    years = ["Year", "base_year"]
    years_list = [str(i) for i in range(1990,int(year)-1)]
    years.extend(years_list)
    years.extend(["change_from_baseline"])

    ### Import each country data
    print("Importing CRF data...")

    ### Create list of folders within data_path/year/data/
    folderlis = sorted([f for f in os.listdir(data_path+year+"/data/") if f.endswith(year)])

    #for k in folderlis:
        #crf_list = sorted(os.listdir(data_path+year+\"/data/\"+k))
    crf_list = sorted([f for f in os.listdir(data_path+year+"/data/UK_NIR_2022") if f.endswith('xlsx')])
    country_name = "UK_NIR_2022".split("_")
    print("...", country_name[0], "CRF data")
        
        ### Read in country data
    xls = pd.ExcelFile(data_path+year+"/data/"+"UK_NIR_2022"+"/"+crf_list[-1])
    crf = pd.read_excel(xls, 'Table10s3', header = 4)
    
    return crf


In [5]:
###############################################################################
### crf_import_n2o
###############################################################################

def crf_import_n2o(year = '2022', data_path = '/Users/ks15697/Documents/Python/UNFCCC/'): 
    import os
    import pandas as pd
    import csv
    import numpy as np

    ### Add folder for processed data to be stored in
    if os.path.isdir(data_path+year+"/processed_data/") == False:
        os.mkdir(data_path+year+"/processed_data/")
        os.mkdir(data_path+year+"/processed_data/totals/")
        os.mkdir(data_path+year+"/processed_data/totals_co2/")
        os.mkdir(data_path+year+"/processed_data/species_specific/")
    if os.path.isdir(data_path+year+"/processed_data/totals/") == False:
        os.mkdir(data_path+year+"/processed_data/totals/")
    if os.path.isdir(data_path+year+"/processed_data/totals_co2/") == False:
        os.mkdir(data_path+year+"/processed_data/totals_co2/")
    if os.path.isdir(data_path+year+"/processed_data/species_specific/") == False:
        os.mkdir(data_path+year+"/processed_data/species_specific/")

    ### Create lists for each gas to go into for country totals

    ### Can this be changed to create lists for each YEAR (headers for Table10s3 are years rather than gases)???

    years = ["Year", "base_year"]
    years_list = [str(i) for i in range(1990,int(year)-1)]
    years.extend(years_list)
    years.extend(["change_from_baseline"])

    ### Import each country data
    print("Importing CRF data...")

    ### Create list of folders within data_path/year/data/
    folderlis = sorted([f for f in os.listdir(data_path+year+"/data/") if f.endswith(year)])

    #for k in folderlis:
        #crf_list = sorted(os.listdir(data_path+year+\"/data/\"+k))
    crf_list = sorted([f for f in os.listdir(data_path+year+"/data/UK_NIR_2022") if f.endswith('xlsx')])
    country_name = "UK_NIR_2022".split("_")
    print("...", country_name[0], "CRF data")
        
        ### Read in country data
    xls = pd.ExcelFile(data_path+year+"/data/"+"UK_NIR_2022"+"/"+crf_list[-1])
    crf = pd.read_excel(xls, 'Table10s4', header = 4)
    
    return crf

In [ ]:
unfccc_scrape(year = '2022', data_path = r'C:/Users/amar1/UNFCCC_data_local/')

In [ ]:
unfccc_unzip(year="2022", data_path = r'C:/Users/amar1/UNFCCC_data_local/')

In [9]:
ch4 = crf_import_ch4(year = '2022', data_path = r'C:/Users/amar1/UNFCCC_data_local/')
ch4

Importing CRF data...
... UK CRF data


,GREENHOUSE GAS SOURCE AND SINK CATEGORIES,Base year(1),1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Change from base to latest reported year
0,NaN,(kt),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%
1,1. Energy,1496.198346,1496.198346,1516.883409,1498.019022,1420.993629,1120.636638,1154.969761,1092.643012,1006.961661,...,384.369606,337.847767,320.953039,303.773576,265.857528,263.057614,262.56135,253.997777,235.556764,-84.256314
2,A. Fuel combustion (sectoral approach),129.43946,129.43946,134.46966,127.281609,130.133233,118.014832,102.281655,102.989162,95.268341,...,43.364553,44.386653,43.105705,45.153607,46.465766,47.673054,49.478277,49.131183,48.218358,-62.748332
3,1. Energy industries,9.327453,9.327453,9.296994,9.501829,9.922269,10.060215,10.488551,10.775702,11.090562,...,11.252326,11.241765,12.737843,13.304238,14.164167,14.788899,15.170421,15.508386,15.282,63.838936
4,2. Manufacturing industries and construction,4.589083,4.589083,4.711231,4.728609,4.377738,4.663597,4.541524,4.210813,3.877838,...,3.38953,3.569429,4.073298,4.273278,4.37072,4.833171,5.192234,5.1501,5.366701,16.944962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Long-term storage of C in waste disposal sites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Indirect N2O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Indirect CO2 (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
n2o = crf_import_n2o(year = '2022', data_path = r'C:/Users/amar1/UNFCCC_data_local/')
n2o

Importing CRF data...
... UK CRF data


,GREENHOUSE GAS SOURCE AND SINK CATEGORIES,Base year(1),1990,1991,1992,1993,1994,1995,1996,1997,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Change from base to latest reported year
0,NaN,(kt),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%
1,1. Energy,12.399834,12.399834,12.32051,12.126234,12.005137,12.267934,12.600877,12.107575,11.725449,...,8.71365,8.628428,8.386046,8.46204,7.961422,7.92665,8.042293,7.921735,7.227956,-41.709255
2,A. Fuel combustion (sectoral approach),12.249575,12.249575,12.174791,11.978783,11.850275,12.075453,12.397266,11.899059,11.539965,...,8.58924,8.510284,8.268197,8.342556,7.838041,7.802034,7.916938,7.797677,7.124227,-41.841029
3,1. Energy industries,4.821621,4.821621,4.773376,4.616961,4.171165,4.055918,3.990189,3.837661,3.603327,...,3.835712,3.716935,3.326995,3.210666,2.553062,2.390532,2.417321,2.39708,2.352618,-51.206911
4,2. Manufacturing industries and construction,1.058335,1.058335,1.052081,1.042331,1.008378,1.046156,1.009588,0.97332,0.921806,...,0.690587,0.63024,0.700639,0.748686,0.754312,0.826021,0.880824,0.837112,0.841022,-20.533441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Long-term storage of C in waste disposal sites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Indirect N2O,14.114362,14.114362,13.794607,13.49868,12.859003,12.717212,12.277937,11.847634,11.073066,...,6.079931,5.759218,5.395422,5.219106,4.753065,4.628073,4.420606,4.182834,3.69858,-73.79563
59,Indirect CO2 (3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
